# 3D Visualization with Plotly

In this notebook, we'll show how to use [plotly](https://plot.ly/python/) to do some 3D visualizations of larcv data.  You may need to install plotly on your machine.

```bash
pip install plotly --user
```

Plotly uses javascript to create nice visualizations to view and manipulate in a web browser.  We'll use its offline mode to keep everything in the notebook.

In [1]:
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

We'll now add the data APIs to our path

In [2]:
import sys
sys.path.append("../")
from osf.image_api import image_reader_3d
from osf.particle_api import *

Welcome to JupyROOT 6.14/04


First, let's load a 3D image.  We'll assume you've read the readme for basic API useage.

In [3]:
fimg = '/gpfs/slac/staas/fs1/g/neutrino/kterao/data/dlprod_ppn_v10/dlprod_192px_00.root'
img_reader = image_reader_3d(fimg)

Let's choose an event to look at

In [6]:
eventn = 48

First, let's look at the energy in each voxel

In [7]:
voxels, energy = img_reader.get_energy(eventn)

voxels and energy are both numpy arrays

In [9]:
voxels

array([[109,  60,   0],
       [110,  60,   0],
       [111,  60,   0],
       ...,
       [ 38, 102, 191],
       [ 39, 102, 191],
       [ 40, 102, 191]], dtype=int32)

In [8]:
energy

array([0.01040413, 0.04662808, 0.01040413, ..., 0.03000904, 0.11896198,
       0.03734245], dtype=float32)

To create plots, we need plotly's graph objects module

In [13]:
import plotly.graph_objs as go
import numpy as np

A histogram of energies indicates a log scale might be reasonable for visualization purposes

In [15]:
fdata = [go.Histogram(x=np.log(energy))]
iplot(fdata)

Here's a function that will create a scatter plot of the energy

In [16]:
def scatter_energy(img_reader, n, threshold=0.0):
    """
    Creates graph object for energy scatter plot
    Args:
        img_reader (image_reader_3d)
        n (int): image to plot
        threshold (optional): threshold energies below a certain value
    """
    voxels, energy = img_reader.get_energy(n)
    inds = energy > threshold
    trace = go.Scatter3d(x=voxels[inds,0], y=voxels[inds,1], z=voxels[inds,2],
                    mode='markers',
                    marker = dict(
                        size = 5,
                        color = np.log(energy[inds]),
                        colorscale='Viridis',
                        opacity=0.8
                    ))
    return trace

note that `iplot` takes a list of graph objects

In [19]:
trace = scatter_energy(img_reader, eventn)
iplot([trace]) # note [] around graph object

We can create a similar function for class of the voxels

In [20]:
def scatter_classes(img_reader, n):
    voxels, types = img_reader.get_classes(n)
    trace = go.Scatter3d(x=voxels[:,0], y=voxels[:,1], z=voxels[:,2],
                    mode='markers',
                    marker = dict(
                        size = 5,
                        color = types,
                        colorscale='Viridis',
                        opacity=0.8
                    ), hovertext=types)
    return trace

In [21]:
trace = scatter_classes(img_reader, eventn)
iplot([trace]) # note [] around graph object

Now, we want to plot the starting location of each particle.  To do this, we need to load the particle file, and use a `particle_reader`

In [22]:
fname = '/gpfs/slac/staas/fs1/g/neutrino/kterao/data/dlprod_ppn_v10/particle/test_particle.root'
preader = particle_reader(fname)

In [25]:
# get start of particle
def get_start(p):
    return p['start_x'], p['start_y'], p['start_z']

# check if particle is in 192 pixel crop
def inbounds(px, py, pz):
    return not(px < 0 or px > 192 or py < 0 or py > 192 or pz < 0 or pz > 192)

# create graph object
def scatter_particles(particle_reader, n):
    particle_info = particle_reader.get_event(n)
    x = []
    y = []
    z = []
    htext = []
    for i in particle_info['particle_idx']:
        p = get_particle(particle_info, i)
        px, py, pz = get_start(p)
        if p['npx'] < 0 or not inbounds(px, py, pz):
            continue
        else:
            htext.append("pdg: %d, E = %f, primary=%d" % (p['pdg_code'], p['creation_energy'], p['primary']))
            x.append(px)
            y.append(py)
            z.append(pz)
        
    x, y, z = np.array(x), np.array(y), np.array(z)
    trace = go.Scatter3d(x=x, y=y, z=z,
                    mode='markers', hovertext=htext)
    return trace

In [26]:
trace = scatter_particles(preader, eventn)
iplot([trace])

If we want to visualize particle starts as well as the point clouds, we can combine traces

In [27]:
trace1 = scatter_classes(img_reader, eventn)
trace2 = scatter_particles(preader, eventn)
iplot([trace1, trace2])